In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path
import statsmodels.api as sm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# --- Load the Data ---
DATA_PATH = Path("../data/SPY_data.csv")

df = pd.read_csv(
    DATA_PATH, 
    index_col=0, 
    parse_dates=True, 
    date_format='%Y-%m-%d'
)

display(df.head())

df.info()

,Close,High,Low,Open,Volume
Price,,,,,
Ticker,SPY,SPY,SPY,SPY,SPY
Date,NaN,NaN,NaN,NaN,NaN
2020-01-02,299.4064636230469,299.4249140098017,297.24987779194856,298.18072230420165,59151200
2020-01-03,297.1392822265625,298.2728840920659,295.93196319223426,295.98725810025013,77709700
2020-01-06,298.2729187011719,298.3558610728196,295.24998004240615,295.3697950655392,55653900


<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, Ticker to 2024-12-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   1259 non-null   object
 1   High    1259 non-null   object
 2   Low     1259 non-null   object
 3   Open    1259 non-null   object
 4   Volume  1259 non-null   object
dtypes: object(5)
memory usage: 59.1+ KB


In [3]:
# --- Data Cleaning Step ---
# 1. Try to convert the 'Close' column to numbers.
#    errors='coerce' will replace any non-numeric values (like 'SPY') with NaN (Not a Number).
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

# 2. Drop any rows that have NaN in the 'Close' column.
rows_before = len(df)
df.dropna(subset=['Close'], inplace=True)
rows_after = len(df)

print(f"Removed {rows_before - rows_after} corrupted row(s).")

# --- Initial Inspection ---
print("\nFirst 5 rows of the cleaned dataset:")
display(df.head())

# Display a summary of the cleaned dataframe
print("\nDataset Info:")
df.info()

Removed 2 corrupted row(s).

First 5 rows of the cleaned dataset:


,Close,High,Low,Open,Volume
Price,,,,,
2020-01-02,299.406464,299.4249140098017,297.24987779194856,298.18072230420165,59151200
2020-01-03,297.139282,298.2728840920659,295.93196319223426,295.98725810025013,77709700
2020-01-06,298.272919,298.3558610728196,295.24998004240615,295.3697950655392,55653900
2020-01-07,297.434174,298.180682883535,296.98256105203933,297.7014229007024,40496400
2020-01-08,299.019470,300.2452118245496,297.3789885629929,297.62781571292214,68296000



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 1258 entries, 2020-01-02 to 2024-12-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1258 non-null   float64
 1   High    1258 non-null   object 
 2   Low     1258 non-null   object 
 3   Open    1258 non-null   object 
 4   Volume  1258 non-null   object 
dtypes: float64(1), object(4)
memory usage: 59.0+ KB


In [4]:
# --- Visualize the Closing Price ---

# This creates an interactive line chart of the 'Close' column.
# The index (our Date column) is automatically used for the x-axis.
fig = px.line(df, y='Close', title='SPY Closing Price Over Time')

# Add range slider for better navigation
fig.update_xaxes(rangeslider_visible=True)

fig.show()

In [5]:
# --- Decompose the Time Series ---

# We'll use an additive model, which assumes that:
# Observed Value = Trend + Seasonality + Residual
# We'll look for a yearly seasonal pattern, so we set the period to 365 days.
decomposition = sm.tsa.seasonal_decompose(df['Close'], model='additive', period=365)

# --- Plot the Decomposition ---

# Create a figure with 4 stacked subplots
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    subplot_titles=("Observed", "Trend", "Seasonal", "Residuals")
)

# Add each component to its own subplot
fig.add_trace(go.Scatter(x=df.index, y=decomposition.observed, mode='lines', name='Observed'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=decomposition.trend, mode='lines', name='Trend'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=decomposition.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index, y=decomposition.resid, mode='lines', name='Residuals'), row=4, col=1)

# Update layout for a cleaner look
fig.update_layout(height=700, title_text="Time Series Decomposition")
fig.show()

In [6]:
# --- Split the Data into Training and Testing Sets (Corrected) ---

# Your logic is correct. We'll use 2020-2023 for training and 2024 for testing.
split_date = '2024-01-01'

train_df = df.loc[df.index < split_date].copy()
test_df = df.loc[df.index >= split_date].copy()

# Plot the split to visualize what we've done
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_df.index, y=train_df['Close'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df.index, y=test_df['Close'], mode='lines', name='Testing Data', line=dict(color='orange')))
fig.update_layout(title='Training and Testing Data Split', xaxis_title='Date', yaxis_title='SPY Close Price')
fig.show()

print(f"Training data contains {train_df.shape[0]} observations.")
print(f"Testing data contains {test_df.shape[0]} observations.")

Training data contains 1006 observations.
Testing data contains 252 observations.


In [7]:
pip install prophet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Users/hardjoshi/Desktop/Code/time-series-forecasting-API/env/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from prophet import Prophet
import pandas as pd

# --- Train the Prophet Model ---

# 1. Prepare the data for Prophet
# Prophet requires columns to be named 'ds' (datestamp) and 'y' (value)
prophet_train_df = train_df.reset_index().rename(columns={'Price': 'ds', 'Close': 'y'})

# 2. Instantiate and fit the model
# Prophet will automatically find the trend and seasonalities
print("Training Prophet model...")
model = Prophet(yearly_seasonality=True, daily_seasonality=False)
model.fit(prophet_train_df)
print("Model training complete.")

# 3. Create a dataframe for future predictions
# This needs to include the dates from our test set.
future = model.make_future_dataframe(periods=len(test_df), freq='D')

# 4. Generate the forecast
print("Generating forecast...")
forecast = model.predict(future)
print("Forecast complete.")

# Display the last few rows of the forecast
display(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

16:39:26 - cmdstanpy - INFO - Chain [1] start processing


Training Prophet model...


16:39:27 - cmdstanpy - INFO - Chain [1] done processing


Model training complete.
Generating forecast...
Forecast complete.


,ds,yhat,yhat_lower,yhat_upper
1253,2024-09-02,512.176500,459.628361,562.615811
1254,2024-09-03,511.765508,459.123258,561.625877
1255,2024-09-04,511.422189,461.145617,558.937771
1256,2024-09-05,510.671459,455.466225,559.556453
1257,2024-09-06,510.207794,456.285808,559.512082


In [9]:
import plotly.graph_objects as go
import pandas as pd

# --- Plot the Forecast (Final Robust Version) ---

# FIX: Explicitly convert both date keys to datetime objects to ensure they match
df.index = pd.to_datetime(df.index)
forecast['ds'] = pd.to_datetime(forecast['ds'])

# Merge the forecast results with the original dataframe
plot_df_final = pd.merge(df, forecast, left_index=True, right_on='ds', how='outer')

# --- Create the figure ---
fig = go.Figure()

# Add the actual historical price
fig.add_trace(go.Scatter(
    x=plot_df_final['ds'], 
    y=plot_df_final['Close'], 
    mode='lines', 
    name='Actual Price',
    line=dict(color='royalblue')
))

# Add the Prophet forecast
fig.add_trace(go.Scatter(
    x=plot_df_final['ds'], 
    y=plot_df_final['yhat'], 
    mode='lines', 
    name='Forecast',
    line=dict(color='orange')
))

# Add the uncertainty interval as a shaded area
fig.add_trace(go.Scatter(
    x=plot_df_final['ds'], y=plot_df_final['yhat_upper'],
    mode='lines', line=dict(width=0),
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=plot_df_final['ds'], y=plot_df_final['yhat_lower'],
    mode='lines', line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(255, 165, 0, 0.2)',
    showlegend=False
))

# Add a vertical line to show where the test period begins
fig.add_vline(x=split_date, line_width=2, line_dash="dash", line_color="red")

fig.update_layout(
    title="Prophet Forecast vs. Actual Price",
    xaxis_title="Date",
    yaxis_title="SPY Close Price"
)
fig.show()

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# --- Calculate Performance Metrics ---

# First, create a dataframe for the test period only and remove any missing values
metrics_df = plot_df_final.loc[plot_df_final['ds'] >= split_date].dropna()

# Calculate the metrics
actuals = metrics_df['Close']
predictions = metrics_df['yhat']

mae = mean_absolute_error(actuals, predictions)
rmse = np.sqrt(mean_squared_error(actuals, predictions))
mape = np.mean(np.abs(predictions - actuals) / np.abs(actuals)) * 100

# Print the results in a user-friendly format
print("--- Forecast Performance Metrics ---")
print(f"Mean Absolute Error (MAE):    ${mae:.2f}")
print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

--- Forecast Performance Metrics ---
Mean Absolute Error (MAE):    $32.66
Root Mean Squared Error (RMSE): $35.72
Mean Absolute Percentage Error (MAPE): 6.28%
